In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# 1. Подготовка датасета

In [ ]:
# Скачиваем датасет с репозитория ЦПУР на Гитхабе
!wget 

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [ ]:
servants = pd.read_csv('declarations_2013_2020_money_pivot.csv', sep = ';')
# всего наблюдений в датасете за 2013-2020 гг.
len(servants)

22201

In [ ]:
# Убираем ненужные для дальнейшего анализа колонки
servants = servants[['state_agency_short', 'year', 'name', 'position', 'position_standard',
                     'position_group', 'married', 'gender', 'children', 'extra', 'income_diff_month_const_чиновник']]


In [ ]:
# проверяем гендерный баланс датасета со всеми чинониками
servants['gender'].value_counts(normalize=True) * 100

m               48.583397
f               47.371740
неопределяем     4.044863
Name: gender, dtype: float64

**Готовим несколько варианов датасета для дальнейшего анализа**

In [ ]:
# оставляем только нужные 7 должностей
posit = ['директор департамента', 'заместитель директора департамента',
         "начальник отдела", 'референт', 'помощник федерального министра', 'советник федерального министра',
         'заместитель федерального министра']

servants_7_pos = servants[servants['position_standard'].isin(posit)]
len(servants_7_pos)

19645

In [ ]:
# Убираем данные Минпросвет и Миннауки как непрезентативные
mins = ['Мипросвещения', 'Миннауки']
servants_7_pos_no_minpros_minnauki = servants_7_pos[~servants_7_pos['state_agency_short'].isin(mins)]
len(servants_7_pos_no_minpros_minnauki)

19194

In [ ]:
# Проверяем гендерный баланс
servants_7_pos_no_minpros_minnauki['gender'].value_counts(normalize=True) * 100

m               50.182349
f               45.962280
неопределяем     3.855371
Name: gender, dtype: float64

**Очиска датасета для проведения регрессионного анализа**

In [ ]:
#Удаляем чиновников с доходами, содержащими ипотеки, субсидии и прочее
servants_7_pos_income = servants_7_pos_no_minpros_minnauki[servants_7_pos_no_minpros_minnauki['extra'] != 'extra']
len(servants_7_pos_income)

18765

Обрезаем выбросы снизу

In [ ]:
# Расчеты сделаны на основе данных, полученных из форм «Обоснования бюджетных
# ассигнований на фонд оплаты труда и страховые взносы в государственные внебюджетные фонды
# в части работников центрального аппарата федеральных государственных органов» ГИС Электронный бюджет.
# Данные были переданы ЦПУР Счетной палатой РФ, в соответствии с решением,
# зафиксированным в письме Минфина России от 24.06.2021 № 01-02-02/21-49981.
# Данные не могут быть размещены в публичном доступе, поэтому здесь мы приводим
# только результаты работы с ними.

# Скачиваем из репозитория ЦПУР за Гитхаб данные окладов по годам с медианными значениями по каждой должности
# Ниже этого чиновник, отработавший полный год, не может получить заработную плату

! wget https://github.com/CAG-ru/cag-public/raw/master/projects/declarations/data/min_oklad.xlsx

min_oklad = pd.read_excel('min_oklad.xlsx')

--2021-12-01 19:51:24--  https://github.com/CAG-ru/cag-public/raw/master/projects/declarations/data/min_oklad.xlsx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CAG-ru/cag-public/master/projects/declarations/data/min_oklad.xlsx [following]
--2021-12-01 19:51:24--  https://raw.githubusercontent.com/CAG-ru/cag-public/master/projects/declarations/data/min_oklad.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10075 (9.8K) [application/octet-stream]
Saving to: ‘min_oklad.xlsx’

min_oklad.xlsx      100%[===================>]   9.84K  --.-KB/s    in 0s      

2021-12-01 19:51:24 (79.8 MB/s) - ‘m

In [ ]:
servants_7_pos_income = servants_7_pos_income.merge(min_oklad, left_on=['year', 'position_standard'],
                            right_on = ['year', 'position_standard'], how='left')

In [ ]:
# Размечаем наблюдения, где среднемесячный доход ниже медианной запрплаты из ОБАСА
servants_7_pos_income['if_lower'] = np.where(servants_7_pos_income['income_diff_month_const_чиновник'] <=  servants_7_pos_income['month_FOT'], 1, 0)

In [ ]:
# Количество случаев, когда доход меньше минимального месячного оклада
len(servants_7_pos_income[servants_7_pos_income['if_lower'] == 1])

649

In [ ]:
# обрезаем выбросы снизу
servants_7_pos_income = servants_7_pos_income[servants_7_pos_income['if_lower'] != 1]

In [ ]:
#Осталось наблюдений
len(servants_7_pos_income)

18116

Обрезаем выбросы сверху

In [ ]:
# Расчитаем размер дохода для 99% персентиля
percentile_99 = servants_7_pos_income['income_diff_month_const_чиновник'].quantile(0.99)
percentile_99

2047959.2968000022

In [ ]:
# 182 наблюдения - выше выбранной границы
over = servants_7_pos_income[servants_7_pos_income['income_diff_month_const_чиновник'] > percentile_99]
len(over)

182

In [ ]:
# Удалим наблюдения выше 99 персентиля
servants_7_pos_income = servants_7_pos_income[servants_7_pos_income['income_diff_month_const_чиновник'] < percentile_99]

In [ ]:
# Количество оставшихся наблюдений
len(servants_7_pos_income)

17933

Проверим гендерное распределение после обработки

In [ ]:
servants_7_pos_income['gender'].value_counts(normalize=True) * 100

m               50.861540
f               45.179278
неопределяем     3.959181
Name: gender, dtype: float64

Убираем наблюдения с неопределенным полом

In [ ]:
# оставим только наблюдения, где пол определен
servants_7_pos_income_gender = servants_7_pos_income[servants_7_pos_income['gender'] != 'неопределяем']
len(servants_7_pos_income_gender)

17223

In [ ]:
# Сохраняем датасет для оценки gender gap и регрессионного анализа
servants_7_pos_income_gender.to_excel('df_declarations_for_regress.xlsx')

# 2. Анализ

**Расчет доли женщин по министерствам**

In [ ]:
# Средняя доля женщин 46%
servants_7_pos_no_minpros_minnauki['gender'].value_counts(normalize=True) * 100

m               50.182349
f               45.962280
неопределяем     3.855371
Name: gender, dtype: float64

In [ ]:
# вычисляем доли м и ж по министерствам (в среднем по всем годам)
gender_share = servants_7_pos_no_minpros_minnauki.groupby(['state_agency_short', 'gender']).size().reset_index()
state_count = servants_7_pos_no_minpros_minnauki.groupby(['state_agency_short']).size().reset_index()
gender_state = gender_share.merge(state_count, left_on=['state_agency_short'], right_on=['state_agency_short'], right_index=False)

gender_state['share'] = round(gender_state['0_x'] / gender_state['0_y'], 3)
gender_state = gender_state[gender_state['gender'] != 'неопределяем']
gender_state

**Расчет доли наблюдений с неопределенным полом по министерствам**

In [ ]:
no_gender = gender_state.groupby(['state_agency_short'])['share'].sum().reset_index()
no_gender['no_gender'] = 1 - no_gender['share']
no_gender

**Без необпределенного пола и с разбивкой на года - динамика доли М и Ж по годам**

In [ ]:
servants_no_gender = servants_7_pos_no_minpros_minnauki[servants_7_pos_no_minpros_minnauki['gender'] != 'неопределяем']

In [ ]:
# делаем расчет с долями м и ж с разбивкой на года
gender_share = servants_no_gender.groupby(['gender', 'year']).size().reset_index()
year_count = servants_no_gender.groupby(['year']).size().reset_index()
gender_year = gender_share.merge(year_count, left_on=['year'], right_on=['year'], right_index=False)

gender_year['share'] = round(gender_year['0_x'] / gender_year['0_y'], 3)
gender_year

**Гендерный баланс по должностям**

In [ ]:
# доли м и ж по должностям (в среднем по всем годам)
gender_share = servants_7_pos_no_minpros_minnauki.groupby(['position_standard', 'gender']).size().reset_index()
state_count = servants_7_pos_no_minpros_minnauki.groupby(['position_standard']).size().reset_index()
g_pos = gender_share.merge(state_count, left_on=['position_standard'], right_on=['position_standard'], right_index=False)

g_pos['share'] = round(g_pos['0_x'] / g_pos['0_y'], 3)
g_pos

**Гендерный баланс по министерставам и группам**

In [ ]:
# доли м и ж по министерствам и группам должностей (в среднем по всем годам)
gender_share = servants_7_pos_no_minpros_minnauki.groupby(['state_agency_short',
                                                            'gender', 'position_group']).size().reset_index()
state_group_count = servants_7_pos_no_minpros_minnauki.groupby(['state_agency_short',
                                                                 'position_group']).size().reset_index()
gender_group = gender_share.merge(state_group_count, left_on=['state_agency_short', 'position_group'],
                             right_on=['state_agency_short', 'position_group'], right_index=False)

gender_group['share'] = round(gender_group['0_x'] / gender_group['0_y'], 3)
gender_group = gender_group[gender_group['gender'] != 'неопределяем']

**Расчет cоотношения доли Ж на должностях высшей группы к доле Ж на должностях главной группы**

In [ ]:
fem = gender_group[gender_group['gender'] == 'f'][['state_agency_short', 'position_group', 'share']].rename(columns={"share": "f_share"})
male = gender_group[gender_group['gender'] == 'm'][['state_agency_short', 'position_group', 'share']].rename(columns={"share": "m_share"})

In [ ]:
group_share = fem.merge(male, left_on=['state_agency_short', 'position_group'],
                  right_on=['state_agency_short', 'position_group'], right_index=False, how='left')

group_high = group_share[group_share['position_group'] == 'высшая'].rename(columns={"f_share": "high_f_share", "m_share": "high_m_share"})
group_main = group_share[group_share['position_group'] == 'главная'].rename(columns={"f_share": "main_f_share", "m_share": "main_m_share"})

gender_index = group_high.merge(group_main, left_on=['state_agency_short'],
                                right_on=['state_agency_short'], right_index=False, how='left')

gender_index['gender_index'] = gender_index['high_f_share'] / gender_index['main_f_share']
gender_index.sort_values('gender_index')

**Гендерный баланс по министерствам, позициям с долей неопределенного пола**

In [ ]:
# делаем расчет с долями м и ж по министерствам-позициям
gender_share_position = servants_7_pos_no_minpros_minnauki.groupby(['state_agency_short', 'gender',
                                                                     'position_standard']).size().reset_index()
pos_year = servants_7_pos_no_minpros_minnauki.groupby(['state_agency_short',
                                                        'position_standard']).size().reset_index()
g_pos = pos_year.merge(gender_share_position, left_on=['state_agency_short', 'position_standard'],
                        right_on=['state_agency_short', 'position_standard'], right_index=False, how='left')

g_pos['share'] = round(g_pos['0_y'] / g_pos['0_x'], 2)


In [ ]:
g_pos_full = g_pos[['state_agency_short', 'position_standard']].drop_duplicates()

In [ ]:
g_non = g_pos[g_pos['gender'] == 'неопределяем']
g_fem = g_pos[g_pos['gender'] == 'f']
g_male = g_pos[g_pos['gender'] == 'm']

In [ ]:
g_pos_non = g_pos_full.merge(g_non, how='left', on=['state_agency_short', 'position_standard'])
g_pos_fem = g_pos_full.merge(g_fem, how='left', on=['state_agency_short', 'position_standard'])
g_pos_male = g_pos_full.merge(g_male, how='left', on=['state_agency_short', 'position_standard'])

In [ ]:
g_pos_non['gender'] = 'неопределяем'
g_pos_fem['gender'] = 'f'
g_pos_male['gender'] = 'm'

In [ ]:
g_pos_all = pd.concat([g_pos_non, g_pos_fem, g_pos_male])
g_pos_all['share'] = g_pos_all['share'].fillna(0)
g_pos_all

**Расчет гендерного разрыва в доходах по министерствам**

In [ ]:
# функция расчета разницы в средних по министерствам

def states_means(df):

  list_states = list(df['state_agency_short'].unique())

  column_names = ['state_mean', 'fem_mean', 'male_mean', 'diff', 'pvalue', 'significance', 'gender_gap']
  res_df_mean = pd.DataFrame(columns = column_names, index = list_states)
                        
  for i in list_states:
    state_mean = df[df['state_agency_short'] == i]['income_diff_month_const_чиновник'].mean()
    fem = df[(df['gender'] == 'f') & (df['state_agency_short'] == i)]
    fem_mean = fem['income_diff_month_const_чиновник'].mean()
    male = df[(df['gender'] == 'm') & (df['state_agency_short'] == i)]
    male_mean = male['income_diff_month_const_чиновник'].mean()
    diff = male_mean - fem_mean
    gender_gap = 1 - round(fem_mean / male_mean, 3)

    t_res = ttest_ind(fem['income_diff_month_const_чиновник'],
                      male['income_diff_month_const_чиновник'], equal_var=False)
    pvalue = t_res.pvalue

    if pvalue <= 0.05: 
      significance = 1
    else:
      significance = 0

    res_df_mean.loc[i] = [state_mean, fem_mean, male_mean, diff, pvalue, significance, gender_gap]

  return res_df_mean

In [ ]:
res = states_means(servants_7_pos_income_gender)
res.sort_values('diff')